In [1]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

sys.path.append('../infra/')
import AssetsData
from Evaluation import Evaluation

In [2]:
df = pd.read_csv("../data/init_data/train.csv").set_index(['TIMESTAMP', 'ID']).sort_index(0)

In [3]:
get = AssetsData.GetData()

In [ ]:
# Get train and test df
train, test = get.get_80_20(df)
train = train.fillna(0)
test  = test.fillna(0)

In [ ]:
def amount_of_days_with_asset(asset_obj, df):
    # Asset is filtered if this thing returns True
    return (df[df.index.get_level_values(1) == asset_obj.name].shape[0] < 300)

def classic_pred_filter(asset_obj, yi_pred):
    return ((yi_pred < asset_obj.factor_stat['y']['mean'] - 5*asset_obj.factor_stat['y']['std']) or
            (yi_pred > asset_obj.factor_stat['y']['mean'] + 5*asset_obj.factor_stat['y']['std']))

asset_pred_df = []

for asset in get.get_assets_names(df):
    print("Current working with {}".format(asset))
    # Regressor to test
    rng = np.random.RandomState(1)
    regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=301, random_state=rng)
    # Asset data
    obj = AssetsData.Asset(asset)
    obj.select_factors(train)
    obj.config_factor_stat(train)
    obj.filter_asset(amount_of_days_with_asset, df)
    obj.train(train, regr, 'AdaboostTreeReg')
    asset_pred_df.append(obj.predict(test, 'AdaboostTreeReg', filter_pred = classic_pred_filter))

Current working with S0000
Current working with S0001
Current working with S0002
Current working with S0003
Current working with S0004
Current working with S0005
Current working with S0006
Current working with S0007
Current working with S0008
Current working with S0009
Current working with S0010
Current working with S0011
Current working with S0012
Current working with S0013
Current working with S0014


In [ ]:
# Concatenate dataframes
ans = pd.concat(asset_pred_df)

In [ ]:
ev = Evaluation()
ev.evaluate(ans)